In [1]:
# === SYSTEM & IMPORTS ===
# Standard-Bibliotheken
import os, sys, json, time, re, glob
from pathlib import Path

# Datenverarbeitung und Mathe
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt

# Machine Learning Frameworks
import tensorflow as tf
from tensorflow import keras

# Scikit-Learn für Metriken und Kalibrierung
from sklearn.metrics import (
    roc_curve, auc, precision_recall_curve, average_precision_score, # Kurven und Scores
    classification_report, confusion_matrix, # Detail-Berichte
    brier_score_loss, # Misst die Qualität der Wahrscheinlichkeiten
    balanced_accuracy_score, matthews_corrcoef # Metriken für unbalancierte Klassen
)
# Tools zur Kalibrierung der Wahrscheinlichkeiten
from sklearn.calibration import calibration_curve, IsotonicRegression
from sklearn.linear_model import LogisticRegression # Für Platt Scaling

# Zum Laden von Objekten (z.B. Scaler)
import joblib, yaml

In [2]:
# === HILFSFUNKTIONEN: LABEL & FILES FINDEN ===
# Da wir Experimente mit verschiedenen Labels (Horizonte, Epsilons) machen,
# müssen wir robust die richtigen Dateien wiederfinden.

# Diese Funktion liest Parameter aus der 'features_v2.yml' (o.ä.)
def label_from_yaml(featureset: str):
    p = f"../data/features_{featureset}.yml"
    if os.path.exists(p):
        with open(p, "r") as f:
            meta = yaml.safe_load(f) or {}
        # Wir suchen den Abschnitt "label"
        lab = (meta.get("label") or {})
        H  = lab.get("horizon")
        md = lab.get("mode")
        eps = lab.get("epsilon")
        # Wenn alles da ist, geben wir es zurück
        if H is not None and md is not None and eps is not None:
            return int(H), str(md), float(eps)
    return None

# Diese Funktion extrahiert Parameter direkt aus Dateinamen wie "..._cls_h5_abs0p0005.csv"
def parse_h_eps_from_path(path: str):
    # Sucht nach "_cls_h(Zahl)_"
    mH = re.search(r"_cls_h(\d+)_", path)
    # Sucht nach Mode und Epsilon am Ende
    me = re.search(r"_(abs|rel|q\d+\.\d+)([\dp.]+)\.csv$", path)
    
    H = int(mH.group(1)) if mH else None
    if me:
        mode, eps_str = me.group(1), me.group(2).replace("p", ".")
        return H, mode, float(eps_str)
    return H, None, None

# Diese Funktion sucht im Datenordner nach der neuesten passenden CSV-Datei
def infer_label_from_files(ticker, interval, start, end, H_hint=None, mode_hint=None, eps_hint=None):
    # Basis-Muster
    pat = f"../data/{ticker}_{interval}_{start}_{end}_cls_h*_.csv".replace("_ .csv",".csv")
    cands = sorted(glob.glob(pat), key=os.path.getmtime)
    # Filter: Muss "_cls_h" im Namen haben
    cands = [c for c in cands if ("_cls_h" in c)]
    
    # Weitere Filter falls Hinweise gegeben sind
    if H_hint is not None:
        cands = [c for c in cands if f"_cls_h{H_hint}_" in c]
    if mode_hint and eps_hint is not None:
        tag = f"{mode_hint}{str(eps_hint).replace('.','p')}"
        cands = [c for c in cands if c.endswith(f"_{tag}.csv")]
        
    if not cands:
        return None
        
    # Wir nehmen die neueste Datei
    return parse_h_eps_from_path(cands[-1])

In [3]:
# === 1) CONFIG & RUN-DIR BESTIMMEN ===
ROOT = os.path.abspath("..")
if ROOT not in sys.path: sys.path.insert(0, ROOT)

# Config laden
with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

TICKER, START, END, INTERVAL = C["ticker"], C["start"], C["end"], C["interval"]
LOOKBACK = int(C["lookback"])
SEED = int(C.get("seed", 42))
FEATURESET = C.get("featureset", "v2")

# Versuchen, Label-Parameter (H/Mode/Epsilon) zu finden
lbl = label_from_yaml(FEATURESET)
if lbl is not None:
    HORIZON, EPS_MODE, EPSILON = lbl
else:
    # Fallback: Raten anhand der Dateien im Ordner
    HORIZON, EPS_MODE, EPSILON = infer_label_from_files(TICKER, INTERVAL, START, END)
    if HORIZON is None or EPS_MODE is None or EPSILON is None:
        raise RuntimeError("Label-Definition (H/mode/epsilon) konnte nicht bestimmt werden. Block 2 nötig.")

print(f"[Block4] Labels: H={HORIZON}, mode={EPS_MODE}, epsilon={EPSILON}")

# Seeds setzen
np.random.seed(SEED); tf.random.set_seed(SEED)

RESULTS_DIR = Path(C.get("results_dir", "../results"))

# Wir suchen den Run-Ordner, der zu diesen Parametern passt (und am neuesten ist)
def _latest_run_dir_matching(results_dir: Path, H: int, eps_mode: str, eps: float) -> Path:
    # Der Tag, den wir im Dateinamen erwarten
    tag = f"{eps_mode}{str(eps).replace('.','p')}"
    # Alle LSTM-Runs holen, sortiert nach Datum (neu -> alt)
    runs = sorted(results_dir.glob("*_lstm"), key=lambda p: p.stat().st_mtime, reverse=True)
    
    for r in runs:
        cfgp = r / "config.json"
        if not cfgp.exists(): continue
        try:
            with open(cfgp, "r") as f:
                rcfg = json.load(f)
            
            # Prüfung: Stimmen Lookback, Horizon und Epsilon überein?
            ok_lb = int(rcfg.get("lookback", LOOKBACK)) == LOOKBACK
            ok_h  = (int(rcfg.get("horizon", H)) == H) or (("_cls_h"+str(H)+"_") in str(rcfg.get("train_csv","")))
            ok_eps= (tag in str(rcfg.get("train_csv","")))
            
            if ok_lb and ok_h and ok_eps:
                return r # Das ist unser Ordner!
        except Exception:
            pass
            
    # Wenn wir keinen perfekten Match finden, nehmen wir zur Not den allerneuesten.
    if runs:
        print("[WARN] Kein exakter Match gefunden, nehme neuesten Run.")
        return runs[0]
        
    raise FileNotFoundError("Kein RUN_DIR gefunden – bitte Block 3 trainieren.")

RUN_DIR = _latest_run_dir_matching(RESULTS_DIR, HORIZON, EPS_MODE, EPSILON)
print("RUN_DIR:", RUN_DIR)

[Block4] Labels: H=1, mode=abs, epsilon=0.0005
RUN_DIR: ..\results\2026-01-03_21-02-35_lstm


In [4]:
# === 2) ARTEFAKTE LADEN (Model, Scaler, Config) ===
# Wir stellen die Trainingsumgebung wieder her.

ENV_INFO = RUN_DIR / "env_info.json"
MODEL_PATH = RUN_DIR / "model.keras"
BEST_PATH  = RUN_DIR / "best.keras" # Das ist der Checkpoint (bestes Val-Ergebnis)
SCALER_PATH = RUN_DIR / "scaler.joblib"
CFG_PATH    = RUN_DIR / "config.json"

# Wir bevorzugen 'best.keras', da 'model.keras' oft nur das Modell der allerletzten Epoche ist (Overfitting-Gefahr).
if BEST_PATH.exists():
    MODEL_PATH = BEST_PATH

# Sicherheitschecks
assert MODEL_PATH.exists(), f"Model-File fehlt: {MODEL_PATH}"
assert SCALER_PATH.exists(), f"Scaler-File fehlt: {SCALER_PATH}"
assert CFG_PATH.exists(),    f"Run-Config fehlt: {CFG_PATH}"

# Config einlesen
with open(CFG_PATH, "r") as f:
    RCFG = json.load(f)

print("Artefakte gefunden.")

Artefakte gefunden.


In [5]:
# === 3) KONSISTENZPRÜFUNG & MODELL-LOAD ===
# Bevor es losgeht, checken wir nochmal, ob die Parameter zusammenpassen.

def _parse_h_mode_eps_from_train_csv(path: str):
    mH = re.search(r"_cls_h(\d+)_", path)
    me = re.search(r"_(abs|rel|q\d+\.\d+)([\dp.]+)\.csv$", path)
    H = int(mH.group(1)) if mH else None
    mode = me.group(1) if me else None
    eps = float(me.group(2).replace("p",".")) if me else None
    return H, mode, eps

run_h_cfg = int(RCFG.get("horizon", HORIZON))
run_lb    = int(RCFG.get("lookback", LOOKBACK))
train_csv_in_cfg = str(RCFG.get("train_csv", ""))

h_from_name, mode_from_name, eps_from_name = _parse_h_mode_eps_from_train_csv(train_csv_in_cfg)

# Wenn der Lookback nicht passt, würde das Modell technisch nicht funktionieren (falsche Input-Shape).
assert run_lb == LOOKBACK, f"Inkompatibler Lookback: run={run_lb} vs. core={LOOKBACK}"

# Warnungen bei kleineren Diskrepanzen (wir brechen nicht hart ab, aber geben Info)
ok_h  = (run_h_cfg == HORIZON) or (h_from_name == HORIZON)
ok_m  = (mode_from_name is None) or (mode_from_name == EPS_MODE)
ok_e  = (eps_from_name  is None) or (np.isclose(eps_from_name, EPSILON))

if not (ok_h and ok_m and ok_e):
    print("[WARN] Run-Config uneindeutig zu Label-Definition, fahre trotzdem fort.")

# Modell laden (ohne Kompilierung, da wir nur Predicten wollen, nicht Trainieren)
model  = keras.models.load_model(MODEL_PATH, compile=False)
# Scaler laden
scaler = joblib.load(SCALER_PATH)

print("Model und Scaler geladen.")

Model und Scaler geladen.


In [6]:
# === 4) ORIGINAL-DATEN LADEN ===
# Wir laden wieder die Feature-CSV, um Testdaten zu generieren.
eps_tag = f"{EPS_MODE}{str(EPSILON).replace('.','p')}"
train_exact = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}_{eps_tag}.csv"

if not os.path.exists(train_exact):
    # Fallback Suche nach Datei, falls der genaue Name abweicht
    pat = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}_{eps_tag}.csv"
    cands = sorted(glob.glob(pat), key=os.path.getmtime)
    if not cands:
        raise FileNotFoundError(f"Train CSV nicht gefunden: {train_exact}")
    TRAIN_CSV = cands[-1]
else:
    TRAIN_CSV = train_exact

print("TRAIN_CSV:", TRAIN_CSV)
# Daten laden
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()

# Konsistenzcheck: Stimmt der Lookback in der Config?
if int(RCFG.get("lookback", LOOKBACK)) != LOOKBACK:
    raise AssertionError("Inkompatibler Lookback!")

TRAIN_CSV: ../data/AAPL_1d_2010-01-01_2026-01-01_cls_h1_abs0p0005.csv


In [7]:
# === 5) FEATURE-SET BESTIMMEN ===
# Wir müssen exakt dieselben Features nutzen wie beim Training.
if "features" in RCFG and RCFG["features"]:
    # Wenn die Liste im Config-JSON steht, nehmen wir sie von dort.
    FEATURES = [c for c in RCFG["features"] if c in df.columns]
else:
    # Sonst laden wir sie aus der YAML.
    with open(f"../data/features_{FEATURESET}.yml","r") as f:
        meta = yaml.safe_load(f) or {}
    FEATURES = [c for c in meta.get("features", []) if c in df.columns]

assert len(FEATURES) > 0, "Keine Features gefunden!"

X = df[FEATURES].copy()
y = df["target"].astype(int).copy()

In [8]:
# === 6) SPLIT WIEDERHERSTELLEN ===
# Wir nutzen exakt denselben Datums-Split wie im Training (Block 3).

# Train: Alles VOR 2024
train_mask = df.index < "2024-01-01"
# Validation: Das Jahr 2024
val_mask   = (df.index >= "2024-01-01") & (df.index < "2025-01-01")
# Test: Das Jahr 2025
test_mask  = df.index >= "2025-01-01"

# Daten aufteilen
X_train, y_train = X.loc[train_mask], y.loc[train_mask]
X_val,   y_val   = X.loc[val_mask],   y.loc[val_mask]
X_test,  y_test  = X.loc[test_mask],  y.loc[test_mask]

print(f"Split sizes: Train={len(X_train)} Val={len(X_val)} Test={len(X_test)}")


Split sizes: Train=3489 Val=252 Test=250


In [9]:
# === 7) SKALIERUNG ANWENDEN ===
# WICHTIG: Wir nutzen scaler.transform, NICHT fit_transform.
# Der Scaler wurde auf Train "gelernt" und muss nun stur angewendet werden.
X_train_s = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=FEATURES)
X_val_s   = pd.DataFrame(scaler.transform(X_val),   index=X_val.index,   columns=FEATURES)
X_test_s  = pd.DataFrame(scaler.transform(X_test),  index=X_test.index,  columns=FEATURES)

In [10]:
# === 8) WINDOWING (Zeitreihen erstellen) ===
# Dieselbe Funktion wie in Block 3 zur Erstellung der Sequenzen.
def make_windows(X_df, y_ser, lookback):
    Xv = X_df.values.astype(np.float32)
    yv = y_ser.values.astype(np.int32)
    xs, ys, idx_end = [], [], []
    for i in range(lookback-1, len(X_df)):
        xs.append(Xv[i - lookback + 1 : i + 1])
        ys.append(yv[i])
        idx_end.append(X_df.index[i]) # Wir speichern das Datum des Labels für den Backtest
    return np.stack(xs, 0), np.array(ys), pd.DatetimeIndex(idx_end)

Xtr_win, ytr, idx_tr = make_windows(X_train_s, y_train, LOOKBACK)
Xva_win, yva, idx_va = make_windows(X_val_s,   y_val,   LOOKBACK)
Xte_win, yte, idx_te = make_windows(X_test_s,  y_test,  LOOKBACK)

# Dataset erstellen für effiziente Prediction
def to_ds(X, y, batch, shuffle):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    # Kein Shuffle bei Test! Reihenfolge ist wichtig.
    if shuffle: ds = ds.shuffle(len(X), seed=SEED)
    return ds.batch(int(C.get("batch", 64))).prefetch(tf.data.AUTOTUNE)

ds_val  = to_ds(Xva_win, yva, int(C.get("batch",64)), shuffle=False)
ds_test = to_ds(Xte_win, yte, int(C.get("batch",64)), shuffle=False)

In [11]:
# === 9) VORHERSAGEN (PREDICTION) ===
# Das Modell gibt Wahrscheinlichkeiten aus (0 bis 1).
y_val_proba  = model.predict(ds_val,  verbose=0).ravel()
y_test_proba = model.predict(ds_test, verbose=0).ravel()

print("Predictions fertig.")

Predictions fertig.


In [12]:
# === 10) KALIBRIERUNG PRÜFEN ===
# Oft sind die Outputs von neuronalen Netzen nicht gut kalibriert.
# Beispiel: Wenn das Modell 0.8 sagt, sollte auch in 80% der Fälle ein Treffer vorliegen.
# Wir testen zwei Kalibrierungsmethoden: Isotonic Regression und Platt Scaling.

# a) Isotonic Regression (flexibel, aber braucht mehr Daten)
iso = IsotonicRegression(out_of_bounds="clip").fit(y_val_proba, yva)
val_iso  = iso.transform(y_val_proba)
test_iso = iso.transform(y_test_proba)

# b) Platt Scaling (Logistische Regression auf den Scores)
platt = LogisticRegression(max_iter=1000)
platt.fit(y_val_proba.reshape(-1,1), yva)
val_platt  = platt.predict_proba(y_val_proba.reshape(-1,1))[:,1]
test_platt = platt.predict_proba(y_test_proba.reshape(-1,1))[:,1]

# Wir messen den "Brier Score Loss" (ähnlich MSE für Wahrscheinlichkeiten).
# Je kleiner, desto besser.
brier_val_raw   = brier_score_loss(yva, y_val_proba)
brier_val_iso   = brier_score_loss(yva, val_iso)
brier_val_platt = brier_score_loss(yva, val_platt)

# Wir wählen den Kandidaten, der auf VALIDATION am besten ist.
if brier_val_platt <= brier_val_iso:
    cand_name, val_cand, test_cand, cand_obj = "platt", val_platt, test_platt, platt
    brier_val_cand = brier_val_platt
else:
    cand_name, val_cand, test_cand, cand_obj = "isotonic", val_iso, test_iso, iso
    brier_val_cand = brier_val_iso

In [13]:
# === 11) KALIBRIERUNG ENTSCHEIDEN ===
# Lohnt sich die Kalibrierung? 
# Wir verlangen eine Mindestverbesserung (min_gain_bp), sonst nehmen wir lieber die Rohdaten (weniger Komplexität).

min_gain_bp = 1.0  # 1 Basispunkt (0.0001)
gain_bp = (brier_val_raw - brier_val_cand) * 1e4

# Nur zur Info: Wie performen sie auf Test? (Darf Entscheidung nicht beeinflussen!)
brier_test_raw  = brier_score_loss(yte, y_test_proba)
brier_test_cand = brier_score_loss(yte, test_cand)

use_cal = (brier_val_raw - brier_val_cand) > 1e-4
if use_cal:
    # Wir nutzen die Kalibrierung
    CAL_METHOD, y_val_cal, y_test_cal = cand_name, val_cand, test_cand
    # Speichern für später
    joblib.dump(cand_obj, RUN_DIR / f"calibrator_{CAL_METHOD}.joblib")
    brier_cal = brier_test_cand
else:
    # Keine Kalibrierung
    CAL_METHOD, y_val_cal, y_test_cal = "none", y_val_proba, y_test_proba
    brier_cal = brier_test_raw

print(f"[Kalibrierung] chosen={CAL_METHOD} | ΔBrier(VAL)={gain_bp:.1f} bp "
      f"| Test Brier raw→cand {brier_test_raw:.4f}→{brier_test_cand:.4f}")

[Kalibrierung] chosen=isotonic | ΔBrier(VAL)=354.4 bp | Test Brier raw→cand 0.2598→0.2543


In [14]:
# === 12) OPTIMALEN THRESHOLD FINDEN ===
# Standard-Schwellwert ist 0.5. Das funktioniert aber nur bei perfekt balancierten Daten gut.
# Wir optimieren den Threshold auf den Validation-Daten, um den MCC zu maximieren.
# MCC ist eine robuste Metrik, die auch bei unbalancierten Klassen gut funktioniert.

def choose_threshold(y_true, y_prob, pos_rate_bounds=(0.45,0.55)):
    # Wir testen alle vorkommenden Wahrscheinlichkeiten als potentielle Thresholds
    uniq = np.unique(y_prob); cand = np.r_[0.0, uniq, 1.0]
    best_t, best_s = 0.5, -1.0
    
    for t in cand:
        yp = (y_prob >= t).astype(int)
        pr = float(yp.mean()) # Positive Rate (Wie oft sagen wir "Kaufen"?)
        
        # Wir wollen keine Extreme (nie handeln oder immer handeln), daher Bounds.
        if not (pos_rate_bounds[0] <= pr <= pos_rate_bounds[1]):
            continue
        
        s = matthews_corrcoef(y_true, yp)
        if s > best_s: best_s, best_t = float(s), float(t)
            
    if best_s < 0:
        return 0.5, 0.0 # Fallback
    return best_t, best_s

# Wir geben enge Grenzen um die tatsächliche Positive-Rate vor.
p_val = float(yva.mean())
bounds = (max(0.0, p_val - 0.10), min(1.0, p_val + 0.10))

# Suche auf Validationset
thr, score_val = choose_threshold(yva, y_val_cal, pos_rate_bounds=bounds)

# Nur zur Info: Was wäre ungebunden (ohne Bounds) oder mit Youden's J passiert?
# (Lassen wir hier im Code stehen, nutzen es aber nicht für die Entscheidung)
def best_mcc_unbounded(y_true, y_prob):
    # ... (vereinfachte Version)
    return 0.0, 0.0 

mcc_raw = 0.0; thr_mcc_raw = 0.0; J_raw = 0.0; thr_J_raw = 0.0 # Platzhalter
print(f"Gewählter Threshold: {thr:.4f} (Validation MCC: {score_val:.3f})")

Gewählter Threshold: 0.5000 (Validation MCC: 0.000)


In [15]:
# === 13) FINALE TEST-EVALUATION (@ chosen Threshold) ===
# Jetzt wird es ernst: Wir wenden den gewählten Threshold auf die Testdaten an.

y_test_pred = (y_test_cal >= thr).astype(int)

# Confusion Matrix: TP, TN, FP, FN
cm   = confusion_matrix(yte, y_test_pred)

# Classification Report: Precision, Recall, F1
rep  = classification_report(yte, y_test_pred, digits=3, output_dict=True)

# AUCs (Area Under Curve) - wichtig unabhängig vom Threshold
fpr, tpr, _ = roc_curve(yte, y_test_cal); roc_auc = auc(fpr, tpr)
prec, rec, _ = precision_recall_curve(yte, y_test_cal); ap = average_precision_score(yte, y_test_cal)

# Baseline für PR-AUC ist die "Naive" Wahrscheinlichkeit (einfach raten)
ap_baseline_test = float(yte.mean())

# Diverse Metriken
bal_acc = balanced_accuracy_score(yte, y_test_pred)
mcc     = matthews_corrcoef(yte, y_test_pred)
pos_rate_test = float(y_test_pred.mean())

# Ausgabe
print("Confusion matrix (test):\n", cm)
print(f"MCC={mcc:.3f} | BalAcc={bal_acc:.3f} | AUROC={roc_auc:.3f} | "
      f"AUPRC={ap:.3f} (baseline={ap_baseline_test:.3f}) | "
      f"thr={thr:.3f} | pred_pos_rate(test)={pos_rate_test:.3f}")

Confusion matrix (test):
 [[ 0 92]
 [ 0 99]]
MCC=0.000 | BalAcc=0.500 | AUROC=0.500 | AUPRC=0.518 (baseline=0.518) | thr=0.500 | pred_pos_rate(test)=1.000


c:\Users\jacin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\jacin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\jacin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(ave

In [16]:
# === 14) BOOTSTRAP KONFIDENZINTREVALL (MCC) ===
# Ein einzelner Wert (z.B. MCC=0.02) sagt wenig aus. Ist das Glück oder Können?
# Wir nutzen Bootstrapping, um ein Konfidenzintervall zu berechnen.
# Dabei sampeln wir Blöcke aus den Testdaten (um Zeitstruktur zu erhalten) und messen MCC erneut.

rng = np.random.default_rng(SEED)
def block_bootstrap_mcc(y_true, y_prob, threshold, n=300, block=LOOKBACK):
    idx = np.arange(len(y_true))
    scores = []
    for _ in range(n):
        # Wir ziehen zufällige Startpunkte für Blöcke
        starts = rng.integers(0, max(1, len(idx)-block+1), size=max(1, len(idx)//block))
        # Wir bauen einen neuen "zusammengewürfelten" Datensatz
        bs = np.concatenate([np.arange(s, min(s+block, len(idx))) for s in starts])
        
        # Wir berechnen MCC auf diesem Sample
        yp = (y_prob[bs] >= threshold).astype(int)
        scores.append(matthews_corrcoef(y_true[bs], yp))
        
    # Wir geben das 2.5%, 50% (Median) und 97.5% Quantil zurück
    return np.percentile(scores, [2.5, 50, 97.5]).astype(float)

mcc_ci = block_bootstrap_mcc(yte, y_test_cal, thr, n=300, block=LOOKBACK)
print("MCC Bootstrap CI [2.5, 50, 97.5]:", [round(x,3) for x in mcc_ci])

MCC Bootstrap CI [2.5, 50, 97.5]: [0.0, 0.0, 0.0]


In [17]:
# === 15) DIAGNOSE-PLOTS SPEICHERN ===
# Visualisierung ist wichtig. Wir speichern Plots im 'figures' Unterordner.
FIG_DIR = RUN_DIR / "figures"; FIG_DIR.mkdir(exist_ok=True, parents=True)

# 1. ROC Curve
plt.figure(figsize=(6,4)); plt.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
plt.plot([0,1],[0,1],"--"); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC (Test)")
plt.legend(); plt.tight_layout(); plt.savefig(FIG_DIR / "roc_test.png", dpi=160); plt.close()

# 2. Precision-Recall Curve
plt.figure(figsize=(6,4))
plt.plot(rec, prec, label=f"AP={ap:.3f} (base={ap_baseline_test:.3f})")
plt.hlines(ap_baseline_test, xmin=0, xmax=1, linestyles="--", label="Random baseline")
plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("Precision-Recall (Test)")
plt.legend(); plt.tight_layout(); plt.savefig(FIG_DIR / "pr_test.png", dpi=160); plt.close()

# 3. Kalibrierungskurve
prob_true, prob_pred = calibration_curve(yte, y_test_cal, n_bins=10, strategy="quantile")
plt.figure(figsize=(6,4)); plt.plot([0,1],[0,1],"--"); plt.plot(prob_pred, prob_true, marker="o")
plt.xlabel("Vorhergesagt"); plt.ylabel("Tatsächlich"); plt.title(f"Kalibrierung (Test) – {CAL_METHOD}")
plt.tight_layout(); plt.savefig(FIG_DIR / "calibration_test.png", dpi=160); plt.close()

# 4. Confusion Matrix
plt.figure(figsize=(4.8,4.2))
plt.imshow(cm, interpolation="nearest"); plt.title("Confusion Matrix (Test)"); plt.colorbar()
ticks = np.arange(2); plt.xticks(ticks, ["0","1"]); plt.yticks(ticks, ["0","1"])
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")
plt.xlabel("Predicted"); plt.ylabel("True")
plt.tight_layout(); plt.savefig(FIG_DIR / "cm_test.png", dpi=160); plt.close()

# 5. Wahrscheinlichkeits-Histogramm
plt.figure(figsize=(6,4))
plt.hist(y_test_proba, bins=30, alpha=0.6, label="raw")
plt.hist(y_test_cal,   bins=30, alpha=0.6, label=f"used ({CAL_METHOD})")
plt.axvline(thr, linestyle="--", label=f"thr={thr:.3f}")
plt.title("P(y=1) – raw vs. used (Test)")
plt.legend(); plt.tight_layout()
plt.savefig(FIG_DIR / "proba_hist_raw_vs_used.png", dpi=160); plt.close()

In [18]:
# === 16) VORHERSAGEN SPEICHERN ===
# Wir speichern die Vorhersagen CSV, falls wir sie extern analysieren wollen.
preds_test = pd.DataFrame({
    "timestamp": idx_te, "y_true": yte,
    "y_proba_raw": y_test_proba, "y_proba_used": y_test_cal, "y_pred": y_test_pred,
}).set_index("timestamp")
preds_test.to_csv(RUN_DIR / "preds_test.csv")

In [19]:
# === 17) EINFACHER BACKTEST (Equity Curves) ===
# Wir simulieren den Erfolg einer Handelsstrategie basierend auf unseren Signalen.

# Reale Kursbewegungen (Log-Returns) für Horizon H
close = df["close"].copy()
fwd_logret = (np.log(close.shift(-HORIZON)) - np.log(close)).reindex(idx_te)

# Wann handeln wir?
signals_t  = (preds_test["y_proba_used"] >= thr).astype(int).reindex(idx_te)

# Realistische Annahme (t+1): Wir bekommen das Signal heute Abend (t) und kaufen morgen früh (t+1).
signals_t1 = signals_t.shift(1).fillna(0) 

# Strategie-Rendite = Signal * Markt-Rendite
strategy_logret_t  = (signals_t  * fwd_logret).fillna(0)
strategy_logret_t1 = (signals_t1 * fwd_logret).fillna(0)

# Kumulierte Rendite (Equity Curve)
equity_t  = strategy_logret_t.cumsum().apply(np.exp) 
equity_t1 = strategy_logret_t1.cumsum().apply(np.exp)

# Vergleich: Buy & Hold (Einfach halten)
bh_logret = (np.log(close.reindex(idx_te)) - np.log(close.reindex(idx_te).iloc[0])).fillna(0)
bh_equity = np.exp(bh_logret)

# KPIs für den Vergleich
def _sharpe(logrets, periods_per_year=252):
    # Sharpe Ratio: Rendite pro Risiko
    mu = logrets.mean() * periods_per_year
    sigma = logrets.std(ddof=1) * np.sqrt(periods_per_year)
    return float(mu / (sigma + 1e-12))

def _cagr(eq, periods_per_year=252):
    # CAGR: Jährliche Wachstumsrate
    T = len(eq) / periods_per_year
    return float((eq.iloc[-1] / eq.iloc[0])**(1.0/max(T,1e-12)) - 1.0)

backtest = {
    "n_trades": int(signals_t.sum()),
    "avg_holding_h": HORIZON,
    "strategy_t":  {"CAGR": _cagr(equity_t),  "Sharpe": _sharpe(strategy_logret_t.dropna()),  "final_equity": float(equity_t.iloc[-1])},
    "strategy_t1": {"CAGR": _cagr(equity_t1), "Sharpe": _sharpe(strategy_logret_t1.dropna()), "final_equity": float(equity_t1.iloc[-1])},
    "buy_hold":    {"CAGR": _cagr(bh_equity), "final_equity": float(bh_equity.iloc[-1])},
}

# Plot speichern
plt.figure(figsize=(8,4))
plt.plot(equity_t.index, equity_t.values,   label="Entry@t (Sofort - unrealistisch)")
plt.plot(equity_t1.index, equity_t1.values, label="Entry@t+1 (Verzögert - realistisch)")
plt.plot(bh_equity.index, bh_equity.values, label="Buy & Hold", linestyle="--")
plt.title(f"Equity Curves (H={HORIZON})")
plt.legend(); plt.tight_layout(); plt.savefig(FIG_DIR / "equity_curves_t_vs_t1.png", dpi=160); plt.close()

In [20]:
# === 18) ALLES ZUSAMMENFASSEN & SPEICHERN ===
# Wir sammeln alle Ergebnisse in einer JSON-Datei.

out = {
    "config": RCFG,
    "features_used": FEATURES,
    "calibration": {
        "chosen": CAL_METHOD,
        "val_brier": {"raw": float(brier_val_raw), "iso": float(brier_val_iso), "platt": float(brier_val_platt)},
    },
    "threshold_selection": {
        "strategy": "max_mcc_with_pos_rate_bounds_centered_on_val_rate",
        "threshold": float(thr),
        "val_mcc": float(score_val),
    },
    "metrics": {
        "test": {
            "roc_auc": float(roc_auc),
            "balanced_accuracy": float(bal_acc),
            "auprc": float(ap),
            "brier": float(brier_cal),
            "mcc": float(mcc),
            "report": rep
        },
        "mcc_bootstrap_ci": [float(x) for x in mcc_ci.tolist()]
    },
    "backtest": backtest
}

with open(RUN_DIR / "evaluation.json", "w") as f:
    json.dump(out, f, indent=2)

print("\nBlock 4 abgeschlossen. Ergebnisse:")
print(" -", RUN_DIR / "evaluation.json")
print(" -", RUN_DIR / "figures")


Block 4 abgeschlossen. Ergebnisse:
 - ..\results\2026-01-03_21-02-35_lstm\evaluation.json
 - ..\results\2026-01-03_21-02-35_lstm\figures


In [21]:
# === 19) EXPLAINABLE AI (SHAP) ===
# Wir nutzen SHAP (SHapley Additive exPlanations), um zu verstehen, welche Features das Modell treiben.
# Da LSTMs auf Sequenzen arbeiten, ist die Interpretation etwas komplexer (Zeit x Feature).

print("\n=== Starte SHAP Analyse ===")
import shap
# Wir unterdrücken TF2 Warnungen für SHAP, falls nötig
# tf.compat.v1.disable_v2_behavior() # Nur aktivieren, falls GradientExplainer Fehler wirft!

# 1. Background Data (Zusammenfassung des Trainingssets)
# Wir können nicht alle Trainingsdaten nutzen (zu langsam), daher nehmen wir eine zufällige Auswahl.
n_background = 100
if len(Xtr_win) > n_background:
    bg_idx = np.random.choice(len(Xtr_win), n_background, replace=False)
    background = Xtr_win[bg_idx]
else:
    background = Xtr_win

# 2. Explainer initialisieren
# GradientExplainer ist gut für TF/Keras Modelle geeignet.
try:
    # Versuche GradientExplainer (erfordert oft Zugriff auf Gradienten im Graph)
    explainer = shap.GradientExplainer(model, background)
    
    # 3. SHAP Values berechnen (für einen Teil des Test-Sets)
    # Auch hier limitieren wir auf eine Auswahl, da sehr rechenintensiv.
    n_shap_test = 50
    if len(Xte_win) > n_shap_test:
        test_idx_shap = np.random.choice(len(Xte_win), n_shap_test, replace=False)
        X_test_sample = Xte_win[test_idx_shap]
    else:
        X_test_sample = Xte_win

    print(f"Berechne SHAP Values für {len(X_test_sample)} Samples (Background {len(background)})...")
    shap_values = explainer.shap_values(X_test_sample)

    # shap_values ist eine Liste von Arrays (eines pro Output-Klasse). Bei Binärklassifikation oft 1 Array.
    if isinstance(shap_values, list):
        vals = shap_values[0]
    else:
        vals = shap_values

    # vals shape: (Samples, Lookback, Features)
    # Wir wollen Feature Importance pro Feature. 
    # Möglichkeit A: Summe über die Zeitachse (Feature ist wichtig, egal wann es auftritt)
    # Möglichkeit B: Letzter Zeitschritt (Feature ist wichtig, wenn es aktuell ist)
    
    # Wir nehmen hier die Summe der absoluten SHAP-Werte über die Zeit, oder einfach die Summe.
    # Für den normalen Summary Plot braucht SHAP (Samples, Features).
    # Wir aggregieren über die Zeitachse (Mittelwert oder Summe).
    shap_avg_over_time = np.sum(vals, axis=1) # (Samples, Features)
    
    # Input Features müssen auch aggregiert werden für die Farbe im Plot (Feature Value high/low)
    # Wir nehmen den Mittelwert der Features über die Zeit als Repräsentant
    X_test_sample_aggr = np.mean(X_test_sample, axis=1)

    # Plot speichern
    plt.figure()
    shap.summary_plot(shap_avg_over_time, X_test_sample_aggr, feature_names=FEATURES, show=False)
    plt.title("SHAP Feature Importance (Summed over Time)")
    plt.tight_layout()
    plt.savefig(FIG_DIR / "shap_summary.png")
    plt.close()
    print(f"SHAP Plot gespeichert unter: {FIG_DIR / 'shap_summary.png'}")

except Exception as e:
    print(f"[ERROR] SHAP Analyse fehlgeschlagen: {e}")
    print("Mögliche Ursache: TF2 Eager Execution Inkompatibilität. Versuche tf.compat.v1.disable_v2_behavior() am Anfang des Notebooks.")


=== Starte SHAP Analyse ===


c:\Users\jacin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Berechne SHAP Values für 50 Samples (Background 100)...


c:\Users\jacin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


SHAP Plot gespeichert unter: ..\results\2026-01-03_21-02-35_lstm\figures\shap_summary.png


C:\Users\jacin\AppData\Local\Temp\ipykernel_40368\816843332.py:59: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_avg_over_time, X_test_sample_aggr, feature_names=FEATURES, show=False)
c:\Users\jacin\AppData\Local\Programs\Python\Python310\lib\site-packages\shap\plots\_beeswarm.py:723: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  summary_legacy(


<Figure size 640x480 with 0 Axes>